# Predicting Wine Quality with Multi-Layer Perceptron (Regression)

### Imports

In [6]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, Dataset, random_split

import matplotlib.pyplot as plt
import pandas as pd
import os

import kagglehub
from pathlib import Path

### Custom models and optimizers

In [ ]:
from models import Wine_MLP     # defined in models.py
from new_optimizers import MomentumSGD_Strong_Wolfe, Adam_Strong_Wolfe, TrustRegionCauchy, Levenberg_Marquardt   # defined in new_optimizers.py

### Downloading the dataset

In [8]:
# download latest version
path = kagglehub.dataset_download("yasserh/wine-quality-dataset")
data_dir = Path(path)
print(list(data_dir.iterdir()))  # see what files are there

[PosixPath('/Users/mikezhang/.cache/kagglehub/datasets/yasserh/wine-quality-dataset/versions/1/WineQT.csv')]


### PyTorch data loading

In [13]:
# PyTorch Dataset for the Wine Quality dataset
class WineDataset(Dataset):
    def __init__(self, csv_file, target_col="quality"):
        df = pd.read_csv(csv_file)

        # features: all columns except the target
        X = df.drop(columns=[target_col]).values
        y = df[target_col].values

        # convert to tensors
        self.X = torch.tensor(X, dtype=torch.float32)
        # use float for regression or long for classification
        self.y = torch.tensor(y, dtype=torch.long)

    def __len__(self):
        return self.X.shape[0]

    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]


In [ ]:
csv_path = data_dir / "WineQT.csv"
full_dataset = WineDataset(csv_path)

# simple train test split
num_samples = len(full_dataset)
num_train = int(0.8 * num_samples)
num_val = num_samples - num_train

train_dataset, val_dataset = torch.utils.data.random_split(
    full_dataset, [num_train, num_val]
)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
val_loader   = DataLoader(val_dataset,   batch_size=64, shuffle=False)

### Writing train and test functions

In [ ]:
criterion = nn.MSELoss()

def train(model, device, train_loader, optimizer, epoch):
    model.train()
    avg_loss = 0.0
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        target = target.float().unsqueeze(1)  # for regression, make sure target shape matches output shape
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()    
        avg_loss += loss.item()

        if batch_idx % 50 == 0:
            print(f'Train Epoch: {epoch} [{batch_idx * len(data)}/{len(train_loader.dataset)}'
                  f' ({100. * batch_idx / len(train_loader):.0f}%)]\tLoss: {loss.item():.6f}')
    
    return avg_loss / len(train_loader)

def validate(model, device, val_loader):
    model.eval()
    val_loss = 0.0
    with torch.no_grad():
        for data, target in val_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            loss = criterion(output, target)
            val_loss += loss.item()
    return val_loss / len(val_loader)

### Model training with the optimizers

In [ ]:
epochs = 5

all_train_loss = []
all_test_loss = []
all_test_acc = []

In [ ]:
# Benchmark 1: SGD with Momentum
print("Benchmark 1: SGD with Momentum")

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = Wine_MLP(input_size=12).to(device)
optimizer1 = torch.optim.SGD(model.parameters(), lr=1e-4, momentum=0.9)

train_loss_over_epochs = []
val_loss_over_epochs = []
for epoch in range(1, epochs + 1):
    avg_loss = train(model, device, train_loader, optimizer1, epoch)
    train_loss_over_epochs.append(avg_loss)
    val_loss = validate(model, device, val_loader)
    val_loss_over_epochs.append(val_loss)

all_train_loss.append(train_loss_over_epochs)
all_val_loss.append(val_loss_over_epochs)
print("Final Validation Loss (SGD with Momentum):", val_loss_over_epochs[-1])

In [ ]:
# Benchmark 2: Adam
print("Benchmark 2: Adam")

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = Wine_MLP(input_size=12).to(device)
optimizer2 = torch.optim.Adam(model.parameters(), lr=1e-3)

train_loss_over_epochs = []
val_loss_over_epochs = []
for epoch in range(1, epochs + 1):
    avg_loss = train(model, device, train_loader, optimizer2, epoch)
    train_loss_over_epochs.append(avg_loss)
    val_loss = validate(model, device, val_loader)
    val_loss_over_epochs.append(val_loss)

all_train_loss.append(train_loss_over_epochs)
all_val_loss.append(val_loss_over_epochs)
print("Final Validation Loss (Adam):", val_loss_over_epochs[-1])

In [ ]:
# New Optimizer 1: MomentumSGD_Strong_Wolfe
print("New Optimizer 1: MomentumSGD_Strong_Wolfe")

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = Wine_MLP(input_size=12).to(device)
optimizer3 = MomentumSGD_Strong_Wolfe(model.parameters(), lr=1e-4, momentum=0.9)

train_loss_over_epochs = []
val_loss_over_epochs = []
for epoch in range(1, epochs + 1):
    avg_loss = train(model, device, train_loader, optimizer3, epoch)
    train_loss_over_epochs.append(avg_loss)
    val_loss = validate(model, device, val_loader)
    val_loss_over_epochs.append(val_loss)
all_train_loss.append(train_loss_over_epochs)
all_val_loss.append(val_loss_over_epochs)
print("Final Validation Loss (MomentumSGD_Strong_Wolfe):", val_loss_over_epochs[-1])


In [ ]:
# New Optimizer 2: Adam_Strong_Wolfe
print("New Optimizer 2: Adam_Strong_Wolfe")

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = Wine_MLP(input_size=12).to(device)
optimizer4 = Adam_Strong_Wolfe(model.parameters(), lr=1e-3)

train_loss_over_epochs = []
val_loss_over_epochs = []
for epoch in range(1, epochs + 1):
    avg_loss = train(model, device, train_loader, optimizer4, epoch)
    train_loss_over_epochs.append(avg_loss)
    val_loss = validate(model, device, val_loader)
    val_loss_over_epochs.append(val_loss)
all_train_loss.append(train_loss_over_epochs)
all_val_loss.append(val_loss_over_epochs)
print("Final Validation Loss (Adam_Strong_Wolfe):", val_loss_over_epochs[-1])

In [ ]:
# New Optimizer 3: TrustRegionCauchy
print("New Optimizer 3: TrustRegionCauchy")

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = Wine_MLP(input_size=12).to(device)
optimizer5 = TrustRegionCauchy(model.parameters(), initial_trust_radius=1.0)

train_loss_over_epochs = []
val_loss_over_epochs = []
for epoch in range(1, epochs + 1):
    avg_loss = train(model, device, train_loader, optimizer5, epoch)
    train_loss_over_epochs.append(avg_loss)
    val_loss = validate(model, device, val_loader)
    val_loss_over_epochs.append(val_loss)
all_train_loss.append(train_loss_over_epochs)
all_val_loss.append(val_loss_over_epochs)
print("Final Validation Loss (TrustRegionCauchy):", val_loss_over_epochs[-1])


In [ ]:
# New Optimizer 4: Levenberg_Marquardt
print("New Optimizer 4: Levenberg_Marquardt")

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = Wine_MLP(input_size=12).to(device)
optimizer6 = Levenberg_Marquardt(model.parameters(), initial_damping=1.0)

train_loss_over_epochs = []
val_loss_over_epochs = []
for epoch in range(1, epochs + 1):
    avg_loss = train(model, device, train_loader, optimizer6, epoch)
    train_loss_over_epochs.append(avg_loss)
    val_loss = validate(model, device, val_loader)
    val_loss_over_epochs.append(val_loss)
all_train_loss.append(train_loss_over_epochs)
all_val_loss.append(val_loss_over_epochs)
print("Final Validation Loss (Levenberg_Marquardt):", val_loss_over_epochs[-1])